In [41]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [34]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv2D,GroupNormalization, BatchNormalization, MaxPool2D, Flatten, Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import gc

In [4]:
data = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
image_path = '/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv'

In [5]:
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [6]:
image_names = data['image_id'].values
labels = data['dx']

In [7]:
image_names

array(['ISIC_0027419', 'ISIC_0025030', 'ISIC_0026769', ...,
       'ISIC_0033536', 'ISIC_0032854', 'ISIC_0032258'], dtype=object)

In [8]:
labels

0          bkl
1          bkl
2          bkl
3          bkl
4          bkl
         ...  
10010    akiec
10011    akiec
10012    akiec
10013    akiec
10014      mel
Name: dx, Length: 10015, dtype: object

In [28]:
df = pd.read_csv(image_path)
images = df.drop(['label'],axis=1)/255
images.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,0.752941,0.600000,0.756863,0.764706,0.607843,0.752941,0.772549,0.603922,0.725490,0.792157,...,0.525490,0.678431,0.486275,0.541176,0.717647,0.576471,0.650980,0.725490,0.603922,0.694118
1,0.098039,0.054902,0.117647,0.266667,0.188235,0.294118,0.482353,0.364706,0.494118,0.619608,...,0.321569,0.235294,0.152941,0.215686,0.098039,0.054902,0.109804,0.098039,0.054902,0.105882
2,0.752941,0.541176,0.600000,0.784314,0.568627,0.639216,0.788235,0.556863,0.627451,0.807843,...,0.584314,0.654902,0.505882,0.560784,0.623529,0.486275,0.556863,0.533333,0.407843,0.458824
3,0.149020,0.074510,0.117647,0.372549,0.231373,0.282353,0.560784,0.403922,0.466667,0.670588,...,0.286275,0.172549,0.101961,0.141176,0.098039,0.047059,0.066667,0.098039,0.047059,0.058824
4,0.619608,0.443137,0.545098,0.760784,0.564706,0.682353,0.843137,0.635294,0.749020,0.882353,...,0.788235,0.819608,0.650980,0.725490,0.674510,0.529412,0.584314,0.427451,0.305882,0.360784


In [10]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

In [11]:
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(images,labels,test_size=0.2, random_state=42)

In [16]:
X_train.shape

(8012, 28, 28, 3)

In [13]:
X_train = X_train.values.reshape(int(len(X_train)),28,28,3)
X_test = X_test.values.reshape(int(len(X_test)),28,28,3)

In [14]:
model = Sequential()

In [29]:
model = Sequential()
model.add(Conv2D(30, (5, 5), strides=(1, 1), padding='valid', activation='relu', input_shape=(28, 28, 3)))
model.add(Conv2D(30, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Conv2D(20, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(Conv2D(15, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(Conv2D(15, (3, 3), strides=(1, 1), padding='valid', activation='relu'))
model.add(GroupNormalization(groups=3))
model.add(MaxPool2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Conv2D(10, (2, 2), strides=(1, 1), padding='valid', activation='relu'))
model.add(Flatten())
model.add(Normalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))


In [30]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [31]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 24, 24, 30)        2280      
                                                                 
 conv2d_19 (Conv2D)          (None, 22, 22, 30)        8130      
                                                                 
 batch_normalization_4 (Batc  (None, 22, 22, 30)       120       
 hNormalization)                                                 
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 11, 11, 30)       0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 9, 9, 20)          5420      
                                                                 
 conv2d_21 (Conv2D)          (None, 7, 7, 15)         

In [32]:
epochs = 50
batch_size = 8

checkpoint_filepath = '/kaggle/working/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy')
EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='EarlyStopping', patience=5)
def scheduler(epoch, lr):
    flr = 1e-3
    if epoch < epochs*0.1 :
        return flr
    elif epoch > epochs*0.1 and epoch < epochs*0.25:
        lr *= tf.math.exp(-0.1)
        return lr
    else:
        lr *= tf.math.exp(-0.008)
#         return lr * tf.math.exp(-0.004)
        return lr
learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


class MemoryClearCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
memory_clear_callback = MemoryClearCallback()

In [35]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[model_checkpoint_callback, EarlyStop, memory_clear_callback], shuffle=True)


Epoch 1/50
1002/1002 [==============================] - 26s 26ms/step - loss: 0.9449 - accuracy: 0.6759 - val_loss: 1.2461 - val_accuracy: 0.5212
Epoch 2/50
1002/1002 [==============================] - 25s 25ms/step - loss: 0.9137 - accuracy: 0.6840 - val_loss: 2.0361 - val_accuracy: 0.6665
Epoch 3/50
1002/1002 [==============================] - 26s 25ms/step - loss: 0.8927 - accuracy: 0.6804 - val_loss: 0.9655 - val_accuracy: 0.6785
Epoch 4/50
1002/1002 [==============================] - 26s 26ms/step - loss: 0.8759 - accuracy: 0.6885 - val_loss: 1.0738 - val_accuracy: 0.6500
Epoch 5/50
1002/1002 [==============================] - 26s 25ms/step - loss: 0.8505 - accuracy: 0.6888 - val_loss: 0.8684 - val_accuracy: 0.6915
Epoch 6/50
1002/1002 [==============================] - 24s 24ms/step - loss: 0.8355 - accuracy: 0.6977 - val_loss: 1.0267 - val_accuracy: 0.6900
Epoch 7/50
1002/1002 [==============================] - 25s 25ms/step - loss: 0.8204 - accuracy: 0.6996 - val_loss: 1.3956 -

In [37]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f%%' % (accuracy * 100))

63/63 [==============================] - 1s 14ms/step - loss: 1.0059 - accuracy: 0.7164
Accuracy: 71.64%
